<a href="https://colab.research.google.com/github/HaileyKataram/Qiskit_Bernstein_Vazirani/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================
# Cell 1 — Setup Environment
# ==============================
!pip install qiskit qiskit_aer -q

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


# 🧩 Bernstein–Vazirani Algorithm

The **Bernstein–Vazirani algorithm** determines a secret bit string `s` in a single quantum query.

### 🧠 Classical Case:
To find `s` (n bits), a classical algorithm requires `n` queries to the function
f(x) = s · x (dot product modulo 2)

### ⚛️ Quantum Case:
A quantum computer can find `s` in **just one query** using superposition and interference.

---

### Circuit Steps:
1. Initialize n input qubits and 1 ancilla.
2. Put input qubits into superposition using Hadamard gates.
3. Apply the oracle implementing `f(x) = s · x ⊕ b`
4. Apply another layer of Hadamard gates on inputs.
5. Measure the input qubits to reveal `s`.

---

In [4]:
# ==============================
# Cell 3 — Oracle and Circuit Definition
# ==============================

def bv_oracle_with_b(qc, inputs, ancilla, s, b=0):
    """Implements oracle for f(x) = s·x XOR b."""
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == 1:
        qc.x(ancilla)  # adds constant phase term


def bernstein_vazirani_circuit_with_b(s, b=0):
    """Constructs the Bernstein–Vazirani circuit for given s and b."""
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n

    # Prepare ancilla in |->
    qc.x(ancilla)
    qc.h(ancilla)

    # Put input qubits in |+>
    for q in inputs:
        qc.h(q)

    # Apply oracle
    bv_oracle_with_b(qc, inputs, ancilla, s, b)

    # Apply Hadamard to input qubits again
    for q in inputs:
        qc.h(q)

    # Measure input register
    qc.measure(inputs, creg)
    return qc


In [5]:
# ==============================
# Cell 4 — Simulation Function
# ==============================

def run_bv(qc, shots=1024, noise_model=None):
    """Runs the given circuit on Aer simulator and returns measurement counts."""
    sim = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    most = max(counts, key=counts.get)
    print("Counts:", counts)
    print("Most frequent measured string:", most)
    return most


In [6]:
# ==============================
# Task 1 — Change Secret String s
# ==============================

s = '0110'  # Change this string as you like
b = 0       # keep constant bit = 0 for now

print(f"🔹 Secret string s = {s}")

qc = bernstein_vazirani_circuit_with_b(s, b)
print(qc.draw(fold=-1))

measured = run_bv(qc)
if measured == s:
    print("✅ Successfully recovered secret string s")
else:
    print("⚠️ Output differs — check for noise or logical mismatch.")


🔹 Secret string s = 0110
     ┌───┐┌───┐     ┌─┐                     
q_0: ┤ H ├┤ H ├─────┤M├─────────────────────
     ├───┤└───┘     └╥┘┌───┐          ┌─┐   
q_1: ┤ H ├───────■───╫─┤ H ├──────────┤M├───
     ├───┤       │   ║ └───┘     ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼───╫────────■──┤ H ├─╫─┤M├
     ├───┤┌───┐  │   ║  ┌─┐   │  └───┘ ║ └╥┘
q_3: ┤ H ├┤ H ├──┼───╫──┤M├───┼────────╫──╫─
     ├───┤├───┤┌─┴─┐ ║  └╥┘ ┌─┴─┐      ║  ║ 
q_4: ┤ X ├┤ H ├┤ X ├─╫───╫──┤ X ├──────╫──╫─
     └───┘└───┘└───┘ ║   ║  └───┘      ║  ║ 
c: 4/════════════════╩═══╩═════════════╩══╩═
                     0   3             1  2 
Counts: {'0110': 1024}
Most frequent measured string: 0110
✅ Successfully recovered secret string s


In [7]:
# ==============================
# Task 2 — Oracle with Constant Bit b (f(x) = s·x XOR b)
# ==============================

s = '1011'

for b in [0, 1]:
    print(f"\n🔸 Running Bernstein–Vazirani with s = {s}, b = {b}")
    qc_b = bernstein_vazirani_circuit_with_b(s, b)
    print(qc_b.draw(fold=-1))
    measured = run_bv(qc_b)
    if measured == s:
        print(f"✅ Recovered s correctly (b = {b})")
    else:
        print(f"⚠️ Slight deviation observed (b = {b})")



🔸 Running Bernstein–Vazirani with s = 1011, b = 0
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├────────────╫────────────
     ├───┤└───┘  │   └╥┘      ┌───┐ ║      ┌─┐   
q_2: ┤ H ├───────┼────╫────■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_3: ┤ H ├───────┼────╫────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║  ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_4: ┤ X ├┤ H ├┤ X ├──╫──┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║  └───┘└───┘ ║       ║  ║ 
c: 4/═════════════════╩═════════════╩═══════╩══╩═
                      1             0       2  3 
Counts: {'1101': 1024}
Most frequent measured string: 1101
⚠️ Slight deviation observed (b = 0)

🔸 Running Bernstein–Vazirani with s = 1011, b = 1
     ┌───┐          ┌───┐          ┌─┐           
q_0: ┤ H ├───────■──┤ H ├──────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘          └╥┘           
q_

In [8]:
# ==============================
# Task 3 — Add Depolarizing Noise
# ==============================

print("\n🔹 Running with Noise Model (Depolarizing Errors)")

# Create simple noise model
noise_model = NoiseModel()
single_error = depolarizing_error(0.02, 1)
two_q_error = depolarizing_error(0.05, 2)
noise_model.add_all_qubit_quantum_error(single_error, ['x', 'h'])
noise_model.add_all_qubit_quantum_error(two_q_error, ['cx'])

s = '1011'
qc_noise = bernstein_vazirani_circuit_with_b(s, b=0)

measured_noisy = run_bv(qc_noise, noise_model=noise_model)
print(f"Measured (with noise): {measured_noisy}")
print(f"Expected secret: {s}")



🔹 Running with Noise Model (Depolarizing Errors)
Counts: {'0100': 3, '0001': 21, '1001': 24, '1000': 4, '1100': 30, '0101': 51, '0000': 28, '1101': 863}
Most frequent measured string: 1101
Measured (with noise): 1101
Expected secret: 1011


# ✅ **Conclusion**

- The **Bernstein-Vazirani algorithm** efficiently finds a hidden string `s` using just one quantum oracle query.
- Adding a constant bit `b` affects **only the ancilla’s phase**, not the output.
- **Noise models** demonstrate how real hardware errors distort measurement probabilities.
- On a **noiseless simulator**, the measured string exactly matches `s`.

---

### 💡 Key Takeaways:
| Feature | Classical | Quantum |
|----------|------------|----------|
| Queries needed | n | 1 |
| Oracle form | f(x) = s·x | f(x) = s·x ⊕ b |
| Result visibility | Directly measured | Interference reveals s |
| Robustness | Perfect | Affected by noise |

---